In [1]:
import numpy as np

In [71]:
import tensorflow as tf
from tensorflow_probability import distributions as tfd

In [72]:
tf_float_type = tf.float64
np_float_type = np.float64
as_tf_float = lambda x: tf.cast(x, tf_float_type)

In [73]:
def tf_logsumexp(ary, axis=1, keepdims=False):
    return tf.math.reduce_logsumexp(ary, axis=axis, keepdims=keepdims)

def tf_logmeanexp(ary, axis=1, keepdims=False):
    return tf.math.reduce_logsumexp(ary, axis=axis, keepdims=keepdims) \
        - tf.math.log(as_tf_float(ary.shape[axis]))

# Model
For $n=1,...,N$,
\begin{align}
\beta \sim N(0,1)\\
x_n \sim N(\beta, 1)
\end{align}

In [34]:
def sample_beta():
    beta = np.random.randn(1)
    return beta

In [35]:
def sample_x(beta, n_sample):
    x = np.random.randn(n_sample) + beta
    return x

In [125]:
def likelihood(mu_beta, beta_sample, x_sample, N):
    """
    Arguments:
    mu_beta: scalar
    beta: array of size [n_MC_out]
    x: array of size [n_MC_out, n_MC_in]
    """
    p_x = tfd.Normal(loc=beta, scale=1)
    p_beta = tfd.Normal(loc=np.float64(0), scale=1)
    q_beta = tfd.Normal(loc=mu_beta, scale=1)
    log_prob_ratio = N*tf.reduce_mean(p_x.log_prob(x_sample), axis=1)\
                    + p_beta.log_prob(beta_sample)\
                    - q_beta.log_prob(beta_sample)
    score = tf_logmeanexp(log_prob_ratio, axis=0)
    return score

In [138]:
beta = sample_beta()
mu_beta=np.float64(1)
N=100000

In [164]:
def d_log_prob(n_MC_out, n_MC_in):
    n_MC_out = 10
    n_MC_in = 10
    q_beta = tfd.Normal(loc=mu_beta, scale=1)
    beta_sample = q_beta.sample([n_MC_out])
    x_sample = sample_x(beta, 2*n_MC_out*n_MC_in).reshape([n_MC_out, 2*n_MC_in])
    x_sample1 = x_sample[:,:n_MC_in ]
    x_sample2 = x_sample[:, n_MC_in:]

    a = likelihood(mu_beta, beta_sample, x_sample1, N)
    b = likelihood(mu_beta, beta_sample, x_sample2, N)
    c = likelihood(mu_beta, beta_sample, x_sample, N)
    return c - 1/2. * (a+b), a, b, c

In [246]:
d_log_prob(10,1)

(<tf.Tensor: shape=(), dtype=float64, numpy=-2.9103830456733704e-11>,
 <tf.Tensor: shape=(), dtype=float64, numpy=-128472.92497394019>,
 <tf.Tensor: shape=(), dtype=float64, numpy=-103082.00833085031>,
 <tf.Tensor: shape=(), dtype=float64, numpy=-115777.46665239528>)

## Thus, the coupling for stochastic VI using Renyi cannot be done by MLMC